### Part 0: Initialisation and saving qHome

In [153]:
import robotic as ry
import time

In [154]:
C = ry.Config()
C.addFile(ry.raiPath('Question-01/environment/cargo.g'))
C.addFile(ry.raiPath('Question-01/environment/cargobot.g'))
C.addFile(ry.raiPath('Question-01/environment/maze.g'))

C.view()
qHome = C.getJointState()
print(qHome)

[ 0.   0.   0.   0.  -0.5  0.  -2.   0.   2.  -0.5]


In [155]:
# del C
# C.view()
# C.getFrameNames()
# qHome = C.getJointState()
# print(qHome)
# print(C.getFrameNames())

### 1.1 Part A: gripper to cargo handle position KOMO_1

In [156]:
# komo_1 = ry.KOMO(C, phases=1, slicesPerPhase=30, kOrder=1, enableCollisions=False)
komo_1 = ry.KOMO()
komo_1.setConfig(C, True)
komo_1.setTiming(1., 20, 5., 0)

# add collision constraints
komo_1.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
komo_1.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

# move the gripper to the cargo position
komo_1.addObjective(
    times=[],
    feature=ry.FS.positionDiff,
    frames=["l_gripper", "cargo_handle"],
    type=ry.OT.eq,
    scale=[1e1]
)

# komo_1.addObjective(
#     times=[],
#     feature=ry.FS.positionDiff,
#     frames=["l_gripper", "cargo_handle"],
#     type=ry.OT.sos,
#     scale=[1e1]
# )

# align the grippper with the cargo_handle
komo_1.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', 'cargo_handle'], ry.OT.eq, [1e1])
# komo_1.addObjective([], ry.FS.scalarProductXY, ['l_panda_base', 'cargo_handle'], ry.OT.eq, [1e1])



ret_1 = ry.NLP_Solver(komo_1.nlp(), verbose=0 ) .solve()
q_gripper_to_cargo = komo_1.getPath()

# for t in range(q_gripper_to_cargo.shape[0]):
#     C.setJointState(q_gripper_to_cargo[t])
#     C.view(False, f'waypoint {t}')
#     time.sleep(0.1)

# C.view()
if ret_1.feasible:
    print("KOMO_1 is feasible")
else:
    print("KOMO_1 is NOT feasible")

KOMO_1 is feasible


### 1.1 Part A: l_panda_base to goal position KOMO_2

In [157]:
komo_2 = ry.KOMO(C, phases=1, slicesPerPhase=20, kOrder=1, enableCollisions=False)
# komo = ry.KOMO()
# komo_2.setConfig(C, True)
# komo_2.setTiming(1., 20, 5., kOrder=1)

# add control objective to stay close to the initial joint configuration
komo_2.addControlObjective([], 0, 1e-1)

# add collision constraints
komo_2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

# Add objective to move cargo to goal area
komo_2.addObjective(
    times=[0., 1.],
    feature=ry.FS.positionDiff,
    frames=["l_panda_base", "goal_area"],
    type=ry.OT.sos
)

ret_2 = ry.NLP_Solver(komo_2.nlp(), verbose=0 ) .solve()
q_cargobot_to_goal_area = komo_2.getPath()



# for t in range(q_cargobot_to_goal_area.shape[0]):
#     C.setJointState(q_cargobot_to_goal_area[t])
#     C.view(False, f'waypoint {t}')
#     time.sleep(0.1)

if ret_2.feasible:
    print("KOMO_2 is feasible")
else:
    print("KOMO_2 is NOT feasible")

KOMO_2 is feasible


### 1.1 Part A: Visualisation of KOMO_1 and KOMO_2

In [158]:
# for t in range(q_gripper_to_cargo.shape[0]):
#     C.setJointState(q_gripper_to_cargo[t])
#     C.view(False, f'waypoint {t}')
#     time.sleep(0.1)
    
# for t in range(q_cargobot_to_goal_area.shape[0]):
#     C.setJointState(q_cargobot_to_goal_area[t])
#     C.view(False, f'waypoint {t}')
#     time.sleep(0.1)

# C.setJointState(qHome)
# C.view()

if ret_1.feasible:
    print("KOMO_1 is feasible")
else:
    print("KOMO_1 is NOT feasible")

if ret_2.feasible:
    print("KOMO_2 is feasible")
else:
    print("KOMO_2 is NOT feasible")

# komo_1.view_play(False, 1)
# komo_2.view_play(False, 1)

KOMO_1 is feasible
KOMO_2 is feasible


### 1.2 Part B: RRT pathfinding (KOMO_1)

In [159]:
ry.params_clear()
ry.params_add({'rrt/stepsize':.5, 'rrt/verbose': 0}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

foundRRTPath = False
numOfTrials = 0

C.setJointState(qHome)


while (not foundRRTPath and numOfTrials < 15):
  rrt = ry.PathFinder()
  rrt.setProblem(C, [qHome], [q_gripper_to_cargo[0]])
  ret_RRT_1 = rrt.solve()
  path = ret_RRT_1.x
  foundRRTPath = ret_RRT_1.feasible
  numOfTrials = numOfTrials + 1
  # print("*** Trial num #", numOfTrials, " result: ", foundRRTPath)
  
print()
print()
print()
print()
print()
print()
print()

if foundRRTPath:
  print("*** Found path at trial num #", numOfTrials, " ***")
else:
  print("** No path found ***")








*** Found path at trial num # 1  ***


In [160]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

### 1.2 Part B: RRT pathfinding (KOMO_2)

In [161]:
ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 0}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

foundRRTPath = False
numOfTrials = 0

C.setJointState(q_gripper_to_cargo[0])

while (not foundRRTPath and numOfTrials < 25):
  rrt = ry.PathFinder()
  rrt.setProblem(C, [q_gripper_to_cargo[0]], [q_cargobot_to_goal_area[0]])
  ret_RRT_2 = rrt.solve()
  path = ret_RRT_2.x
  foundRRTPath = ret_RRT_2.feasible
  numOfTrials = numOfTrials + 1
  # print("*** Trial num #", numOfTrials, " result: ", foundRRTPath)
  
print()
print()
print()
print()
print()
print()
print()

if foundRRTPath:
  print("*** Found path at trial num #", numOfTrials, " ***")
else:
  print("** No path found ***")








*** Found path at trial num # 1  ***


In [162]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [ ]:
# C.attach("l_gripper", "cargo_handle")
# del komo
# C.getFrame("base_origin").setPosition([1.7, -1.6, 0.1])
# C.getFrame("base_origin").setQuaternion([1, 0, 0, 0])
# C.view()